## Curate data

#### Read json files

In [5]:
import os
import json
from pathlib import Path

# Point to raw data folder
repo_root = Path.cwd().parent if Path.cwd().name == "notebook" else Path.cwd()
raw_dir   = os.path.join(repo_root, "data", "raw")

# List all files
all_files = os.listdir(raw_dir)

# Filter for receipts and items JSON
receipt_files = [f for f in all_files if f.startswith("receipts_") and f.endswith(".json")]
item_files = [f for f in all_files if f.startswith("items_") and f.endswith(".json")] 

# Validate
if not receipt_files:
    raise FileNotFoundError(f"No receipts_*.json found in {raw_dir}")
if not item_files:
    raise FileNotFoundError(f"No items_*.json found in {raw_dir}")

#Read the files 
receipt_filename = receipt_files[0]
item_filename    = item_files[0]

receipt_path = os.path.join(raw_dir, receipt_filename)
item_path    = os.path.join(raw_dir, item_filename)

print("→ Loading receipts from", receipt_filename)
print("→ Loading items    from", item_filename)

# Load them (with UTF-8)
with open(receipt_path, "r", encoding="utf-8") as f:
    receipts = json.load(f)

with open(item_path, "r", encoding="utf-8") as f:
    items = json.load(f)

print(f"✅ Loaded {len(receipts)} receipts, {len(items)} items")


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'c:\\Users\\abner\\Desktop\\POS_API_Pipeline\\notebooks\\data\\raw'

#### Create table with desired content

In [1]:
#Transform into a flat table
rows = []
for r in receipts:
    date_time = r["receipt_date"]
    date, time_z = date_time.split("T")
    time = time_z.rstrip("Z")
    payment_types = ";".join(p["type"] for p in r.get("payments", []))

    for line in r.get("line_items", []):
        mods = ";".join(f"{m['name']}({m['option']})"
                        for m in line.get("line_modifiers", [])) or None

        rows.append({
            "receipt_number": r["receipt_number"],
            "date":           date,
            "time":           time,
            "order_type":     r.get("order"),
            "item_name":      line.get("item_name"),
            "cost":           line.get("cost"),
            "modifiers":      mods,
            "payment_type":   payment_types
        })

df = pd.DataFrame(rows)
print(df.head())

NameError: name 'receipts' is not defined

### Quick analysis